In [1]:
# simple implementation of functools.singledispatch
def singledispatch(fn):
    registry = {}
    
    registry[object] = fn
    
    def decorator(arg):
        return registry.get(type(arg), registry[object])(arg)
    
    def register(type_):
        
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner   
    
    def dispatch(type_):
        return registry.get(type_, registry[object])
    
    decorator.register = register
    decorator.dispatch = dispatch
    return decorator

In [2]:
@singledispatch
def htmlize(a):
    from html import escape
    return escape(str(a))

In [3]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

@htmlize.register(str)
def html_str(a):
    return str(a).replace('\n', '</br>')

In [4]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_seq(seq):
    items = ('<li>{0}</li>'.format(htmlize(item)) 
            for item in seq
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [5]:
htmlize(100)

'100(<i>0x64</i>)'

In [6]:
htmlize("""
this is a 
newline
""")

'</br>this is a </br>newline</br>'

In [7]:
htmlize([100, (1, 2, 3), 'string'])

'<ul>\n<li>100(<i>0x64</i>)</li>\n<li><ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul></li>\n<li>string</li>\n</ul>'

In [8]:
htmlize.dispatch(int)

<function __main__.html_int>

# functools.singledispatch

In [9]:
from functools import singledispatch

```python
def singledispatch(func):
    """Single-dispatch generic function decorator.

    Transforms a function into a generic function, which can have different
    behaviours depending upon the type of its first argument. The decorated
    function acts as the default implementation, and additional
    implementations can be registered using the register() attribute of the
    generic function.

    """
    registry = {}
    dispatch_cache = WeakKeyDictionary()
    cache_token = None

    def dispatch(cls):
        """generic_func.dispatch(cls) -> <function implementation>

        Runs the dispatch algorithm to return the best available implementation
        for the given *cls* registered on *generic_func*.

        """
        nonlocal cache_token
        if cache_token is not None:
            current_token = get_cache_token()
            if cache_token != current_token:
                dispatch_cache.clear()
                cache_token = current_token
        try:
            impl = dispatch_cache[cls]
        except KeyError:
            try:
                impl = registry[cls]
            except KeyError:
                impl = _find_impl(cls, registry)
            dispatch_cache[cls] = impl
        return impl

    def register(cls, func=None):
        """generic_func.register(cls, func) -> func

        Registers a new implementation for the given *cls* on a *generic_func*.

        """
        nonlocal cache_token
        if func is None:
            return lambda f: register(cls, f)
        registry[cls] = func
        if cache_token is None and hasattr(cls, '__abstractmethods__'):
            cache_token = get_cache_token()
        dispatch_cache.clear()
        return func

    def wrapper(*args, **kw):
        return dispatch(args[0].__class__)(*args, **kw)

    registry[object] = func
    wrapper.register = register
    wrapper.dispatch = dispatch
    wrapper.registry = MappingProxyType(registry)
    wrapper._clear_cache = dispatch_cache.clear
    update_wrapper(wrapper, func)
    return wrapper
```